# FIXTURE UPDATE  
Obrigatorio passar LEAGUE_ID e SESON  
Atualizado:
<ul>
  <li>Referee</li>
  <li>Status</li>
  <li>Fixture</li>
</ul>

> #TODO  
>  
> -> Hoje os IDs dos times entao sendo inseridos da forma que vem, temos que validar se aquele id de time, é o mesmo ID na tabela de **teams**  
> -> Jogar resultset em um dict, tratar e fazer um insert mais limpo  
> .


In [17]:
pip install ipython_genutils

Note: you may need to restart the kernel to use updated packages.


In [18]:
import ipynb.fs.full.FootballApi as api
import ipynb.fs.full.PostgreDB as pg
import json
import ipython_genutils
from datetime import datetime

In [19]:
#paranaesne
league = 606
#season - Em caso de ligas que inician em um ano e finalizam em outro, utilizar o ano de inicio
season = 2022

In [20]:
# atualizando rounds
%run ./round_updater.ipynb

185
{'get': 'fixtures/rounds', 'parameters': {'season': '2022', 'league': '606'}, 'errors': [], 'results': 11, 'paging': {'current': 1, 'total': 1}, 'response': ['Group Stage - 1', 'Group Stage - 2', 'Group Stage - 3', 'Group Stage - 4', 'Group Stage - 5', 'Group Stage - 6', 'Group Stage - 7', 'Group Stage - 8', 'Group Stage - 9', 'Group Stage - 10', 'Group Stage - 11']}


In [21]:
conn = pg.PostgreDB.connect()

In [22]:
#importando fixture
fixtures = api.FootballApi(function=f"fixtures?league={league}&season={season}").get_data()

In [47]:

my_json = fixtures['response'][0]
print(json.dumps(my_json, indent=4))

#print(fixtures['response'][0])

{
    "fixture": {
        "id": 814566,
        "referee": "Andr\u00e9 Felipe Oliv\u00e9rio",
        "timezone": "UTC",
        "date": "2022-01-22T19:00:00+00:00",
        "timestamp": 1642878000,
        "periods": {
            "first": 1642878000,
            "second": 1642881600
        },
        "venue": {
            "id": 230,
            "name": "Est\u00e1dio Major Ant\u00f4nio Couto Pereira",
            "city": "Curitiba, Paran\u00e1"
        },
        "status": {
            "long": "Match Finished",
            "short": "FT",
            "elapsed": 90
        }
    },
    "league": {
        "id": 606,
        "name": "Paranaense - 1",
        "country": "Brazil",
        "logo": "https://media.api-sports.io/football/leagues/606.png",
        "flag": "https://media.api-sports.io/flags/br.svg",
        "season": 2022,
        "round": "Group Stage - 1"
    },
    "teams": {
        "home": {
            "id": 147,
            "name": "Coritiba",
            "logo": "htt

In [57]:
for fixture in fixtures['response']:

    fixture_id = fixture["fixture"]['id']
    fixture_referee = fixture["fixture"]['referee']
    fixture_timezone = fixture["fixture"]['timezone']
    fixture_date = fixture["fixture"]['date']
    fixture_timestamp = datetime.utcfromtimestamp(fixture["fixture"]['timestamp'])
    
    fixture_periods = fixture["fixture"]['periods']
    if fixture_periods['first']:
        fixture_first_half_start = datetime.utcfromtimestamp(fixture_periods['first'])
    else:
        fixture_first_half_start = datetime.fromisoformat('1900-01-01T00:00:00')
        
    if fixture_periods['second']:
        fixture_second_half_start = datetime.utcfromtimestamp(fixture_periods['second'])
    else:
        fixture_second_half_start = datetime.fromisoformat('1900-01-01T00:00:00')

    team_home = fixture["teams"]['home']
    team_home_id = team_home['id']
    team_home_name = team_home['name']
    team_home_winner = team_home['winner'] if team_home['winner'] is not None else False
    team_home_goals = fixture["goals"]['home'] if fixture["goals"]['home'] is not None else 0

    team_away = fixture["teams"]['away']
    team_away_id = team_away['id']
    team_away_name = team_away['name']
    team_away_winner = team_away['winner'] if team_away['winner'] is not None else False
    team_away_goals = fixture["goals"]['away'] if fixture["goals"]['away'] is not None else 0

dict = {
     'team_home_ht_goals': None,
     
}

    team_detail_score = fixture["score"]
    team_home_ht_goals = team_detail_score["halftime"]['home'] if team_detail_score["halftime"]['home'] is not None else 0
    team_home_ft_goals = team_detail_score["fulltime"]['home'] if team_detail_score["fulltime"]['home'] is not None else 0
    team_home_extratime_goals = team_detail_score["extratime"]['home'] if team_detail_score["extratime"]['home'] is not None else 0
    team_home_penalty_goals = team_detail_score["penalty"]['home'] if team_detail_score["penalty"]['home'] is not None else 0
    team_away_ht_goals = team_detail_score["halftime"]['away'] if team_detail_score["halftime"]['away'] is not None else 0
    team_away_ft_goals = team_detail_score["fulltime"]['away'] if team_detail_score["fulltime"]['away'] is not None else 0
    team_away_extratime_goals = team_detail_score["extratime"]['away'] if team_detail_score["extratime"]['away'] is not None else 0
    team_away_penalty_goals = team_detail_score["penalty"]['away'] if team_detail_score["penalty"]['away'] is not None else 0

    status = fixture["fixture"]['status']
    fixture_status_long = status['long']
    fixture_status_short = status['short']

    # apenas para validacao
    league_id = fixture["league"]['id'] if fixture["league"]['id'] is not None else 'NULL'
    stadium = fixture["fixture"]['venue']
    stadium_id = stadium['id'] if stadium['id'] is not None else 'NULL'


    # ---------------------------------------------------------------------
    # REFEREE
    if fixture_referee:
        sql = f"""INSERT INTO public.referee (name) VALUES('{fixture_referee}') 
                    ON CONFLICT DO NOTHING 
                    RETURNING id """
        cur1 = pg.PostgreDB.run_query(conn, sql)
        res1 = cur1.fetchone()
        if res1:
            referee_id = res1[0]
        else:
            sql = f"SELECT id FROM public.referee where name='{fixture_referee}'"
            print(sql)
            cur2 = pg.PostgreDB.run_query(conn, sql)
            res2 = cur2.fetchone()
            if res2:
                referee_id = res2[0]
            else:
                referee_id = 'NULL'
    else:
        referee_id = 'NULL'

    # ---------------------------------------------------------------------
    # STATUS

    if fixture_status_long:
        sql = f"""INSERT INTO status (long_status, short_status) VALUES('{fixture_status_long}','{fixture_status_short}') 
                    ON CONFLICT DO NOTHING 
                    RETURNING id """
        cur12 = pg.PostgreDB.run_query(conn, sql)
        res12 = cur12.fetchone()
        if res12:
            status_id = res12[0]
        else:
            sql = f"SELECT id FROM status where long_status='{fixture_status_long}'"
            cur22 = pg.PostgreDB.run_query(conn, sql)
            res22 = cur22.fetchone()
            if res22:
                status_id = res22[0]
            else:
                status_id = 'NULL'
    else:
        status_id = 'NULL'
    
    # ---------------------------------------------------------------------
    # FIXTURE/PARTIDA

    sql = f""" INSERT INTO fixture
                    (id, 
                    id_league, 
                    id_referee, 
                    id_stadium, 
                    schedule_timestamp, 
                    first_period_start_timestamp, 
                    second_period_start_timestamp, 
                    id_status, 
                    id_home_team, 
                    home_winner, 
                    home_goals, 
                    id_away_team, 
                    away_winner, 
                    away_goals, 
                    ht_home_goals,
                    ht_away_goals, 
                    ft_home_goals, 
                    ft_away_goals, 
                    et_home_goals, 
                    et_away_goals, 
                    penalty_home, 
                    penalty_away)
                VALUES({fixture_id}, 
                    {league_id}, 
                    {referee_id}, 
                    {stadium_id}, 
                    '{fixture_timestamp}', 
                    '{fixture_first_half_start}', 
                    '{fixture_second_half_start}',
                    {status_id}, 
                    {team_home_id}, 
                    {team_home_winner}, 
                    {team_home_goals}, 
                    {team_away_id}, 
                    {team_away_winner}, 
                    {team_away_goals}, 
                    {team_home_ht_goals}, 
                    {team_away_ht_goals}, 
                    {team_home_ft_goals}, 
                    {team_away_ft_goals}, 
                    {team_home_extratime_goals}, 
                    {team_away_extratime_goals}, 
                    {team_home_penalty_goals}, 
                    {team_away_penalty_goals})
                ON CONFLICT DO NOTHING
            """
    print(sql)
    pg.PostgreDB.run_query(conn, sql)


pg.PostgreDB.commit(conn)
pg.PostgreDB.close(conn)

SELECT id FROM public.referee where name='André Felipe Olivério'
 INSERT INTO fixture
                    (id, 
                    id_league, 
                    id_referee, 
                    id_stadium, 
                    schedule_timestamp, 
                    first_period_start_timestamp, 
                    second_period_start_timestamp, 
                    id_status, 
                    id_home_team, 
                    home_winner, 
                    home_goals, 
                    id_away_team, 
                    away_winner, 
                    away_goals, 
                    ht_home_goals,
                    ht_away_goals, 
                    ft_home_goals, 
                    ft_away_goals, 
                    et_home_goals, 
                    et_away_goals, 
                    penalty_home, 
                    penalty_away)
                VALUES(814566, 
                    606, 
                    321, 
                    230, 
                

In [56]:
conn = pg.PostgreDB.connect()